In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import polars as pl
import toml
from support import preprocess

In [3]:
_base_pth = toml.load("data_path.toml")['files']['pth']
_folder_name = 'normal_ss_t1'

_data_pth = os.path.join(_base_pth, _folder_name)

_mobbo_base_names = [
    'mobbo1.csv',
    'mobbo2.csv',
    'mobbo3.csv',
    'mobbo4.csv',
    'mobbo5.csv',
    'mobbo6.csv',
                     ]

In [4]:
dfs = {'m1':pl.read_csv(os.path.join(_data_pth, _mobbo_base_names[0])),
       'm2':pl.read_csv(os.path.join(_data_pth, _mobbo_base_names[1])),
       'm3':pl.read_csv(os.path.join(_data_pth, _mobbo_base_names[2])),
       'm4':pl.read_csv(os.path.join(_data_pth, _mobbo_base_names[3])),
       'm5':pl.read_csv(os.path.join(_data_pth, _mobbo_base_names[4])),
       'm6':pl.read_csv(os.path.join(_data_pth, _mobbo_base_names[5]))}

## Changing polarity for two boards m5 and m6

In [5]:
dfs['m5'] = dfs['m5'].with_columns([(pl.col('f1')*-1).alias('f1'),
                                    (pl.col('f2')*-1).alias('f2'),
                                    (pl.col('f3')*-1).alias('f3'),
                                    (pl.col('f4')*-1).alias('f4'),])

dfs['m6'] = dfs['m6'].with_columns([(pl.col('f1')*-1).alias('f1'),
                                    (pl.col('f2')*-1).alias('f2'),
                                    (pl.col('f3')*-1).alias('f3'),
                                    (pl.col('f4')*-1).alias('f4'),])

## Pre processing files

In [13]:
# trunkate dfs
_temp_df = dfs['m1']
_start_inx = None
_end_inx = None
_trailing_trigger = False
for i in range(len(_temp_df['sync'])):
    if (_temp_df['sync'][i] == 1) and (not _trailing_trigger):
        _start_inx = i
        _trailing_trigger = True
    
    if (_temp_df['sync'][i] == 0) and (_trailing_trigger):
        _end_inx = i
        break
    
_temp_df = _temp_df[_start_inx:_end_inx]       
        

In [16]:
_temp_df.columns[1:-1]

['f1', 'f2', 'f3', 'f4']

In [20]:
from scipy.interpolate import interp1d

_reference_df = dfs['m1']

_target_dfs = [dfs['m2'], dfs['m3'], dfs['m4'], dfs['m5'], dfs['m6']]
columns = _reference_df.columns[1:-1]

interp_func = []
for _col in columns:
    interp_func.append(interp1d(_reference_df['timestamp'], _reference_df[columns], kind='linear', axis=0, fill_value='extrapolate'))

In [21]:
interp_func